In [53]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [54]:
df = pd.read_csv("../data/raw/train.csv")

In [55]:
df_test = pd.read_csv("../data/raw/test.csv")


In [56]:
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [57]:
mapa_satisfaccion = {'neutral or dissatisfied': 0, 'satisfied': 1}
mapa_clase = {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
mapa_viaje = {'Personal Travel': 0, 'Business travel': 1}

In [58]:
for dataset in [df, df_test]:
    # Limpieza de espacios (solo si es texto)
    # Convertimos a números usando map
    dataset['satisfaction'] = dataset['satisfaction'].astype(str).str.strip().map(mapa_satisfaccion)
    dataset['Class'] = dataset['Class'].astype(str).str.strip().map(mapa_clase)
    dataset['Type of Travel'] = dataset['Type of Travel'].astype(str).str.strip().map(mapa_viaje)

In [59]:
df = df.drop(columns=['Unnamed: 0', 'id', 'Departure Delay in Minutes'])
df_test = df_test.drop(columns=['Unnamed: 0', 'id', 'Departure Delay in Minutes', 'Gender'])

In [65]:
gender_dummies = pd.get_dummies(df['Customer Type'], prefix='Customer Type')
df = pd.concat([df.drop(columns=['Gender']), gender_dummies], axis=1)

gender_dummies = pd.get_dummies(df_test['Customer Type'], prefix='Customer Type')
df_test = pd.concat([df_test.drop(columns=['Gender']), gender_dummies], axis=1)

df = df.replace({True : 1, False : 0})

KeyError: "['Gender'] not found in axis"

In [61]:
print("Valores únicos en Class:", df['Class'].unique())  # Debería salir [1 2 0] o similar, NO nan
print(df.info())

Valores únicos en Class: [1 2 0]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Customer Type                      103904 non-null  object 
 1   Age                                103904 non-null  int64  
 2   Type of Travel                     103904 non-null  int64  
 3   Class                              103904 non-null  int64  
 4   Flight Distance                    103904 non-null  int64  
 5   Inflight wifi service              103904 non-null  int64  
 6   Departure/Arrival time convenient  103904 non-null  int64  
 7   Ease of Online booking             103904 non-null  int64  
 8   Gate location                      103904 non-null  int64  
 9   Food and drink                     103904 non-null  int64  
 10  Online boarding                    103904 non-null  int64  
 11  Seat c

In [62]:
df.head()

,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,...,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Arrival Delay in Minutes,satisfaction,Customer Type_Loyal Customer,Customer Type_disloyal Customer
0,Loyal Customer,13,0,1,460,3,4,3,1,5,...,4,3,4,4,5,5,18.0,0,1,0
1,disloyal Customer,25,1,2,235,3,2,3,3,1,...,1,5,3,1,4,1,6.0,0,0,1
2,Loyal Customer,26,1,2,1142,2,2,2,2,5,...,4,3,4,4,4,5,0.0,1,1,0
3,Loyal Customer,25,1,2,562,2,5,5,5,2,...,2,5,3,1,4,2,9.0,0,1,0
4,Loyal Customer,61,1,2,214,3,3,3,3,4,...,3,4,4,3,3,3,0.0,1,1,0


In [63]:
df_test.head() 

,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,...,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Arrival Delay in Minutes,satisfaction,Customer Type_Loyal Customer,Customer Type_disloyal Customer
0,52,1,0,160,5,4,3,4,3,4,...,5,5,5,2,5,5,44.0,1,True,False
1,36,1,2,2863,1,1,3,1,5,4,...,4,4,4,3,4,5,0.0,1,True,False
2,20,1,0,192,2,0,2,4,2,2,...,4,1,3,2,2,2,0.0,0,False,True
3,44,1,2,3377,0,0,0,2,3,4,...,1,1,1,3,1,4,6.0,1,True,False
4,49,1,0,1182,2,3,4,3,4,1,...,2,2,2,4,2,4,20.0,1,True,False
